# Извлечение текстовой информации из диалогов сотрудников

Нужно разработать скрипт для парсинга диалогов.

Задачи, которые должен выполнять скрипт:

1. Извлекать реплики с приветствием – где менеджер поздоровался. 
2. Извлекать реплики, где менеджер представил себя. 
3. Извлекать имя менеджера. 
4. Извлекать название компании. 
5. Извлекать реплики, где менеджер попрощался.
6. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

# Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier, Pool, cv

In [2]:
pd.options.display.float_format = '{:0.3f}'.format
pd.options.mode.chained_assignment = None

%config InlineBackend.figure_format = 'retina'

Загрузим диалоги.

In [3]:
test_data = pd.read_csv('test_data.csv')
# Имена в диалогах начинаются со строчной буквы, поэтому нет смысла сохранять регистр.
test_data.text = test_data.text.str.lower()

print(test_data.shape)
test_data.head()

(480, 4)


,dlg_id,line_n,role,text
0,0,0,client,алло
1,0,1,manager,алло здравствуйте
2,0,2,client,добрый день
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,ага


Нас интересуют только role - manager, кроме того очевидно что извлекаемая информация содержится в первых или в последних строках.

* В обнавленной таблице отлично видны данные, которые требуется извлечь.

In [4]:
def agg_func(group):
    d = {}
    arr = [x for x in range(2)]
    for i in arr + [-x-1 for x in arr[::-1]]:
        d['phrase_' + str(i)] = group.iloc[i, 1]
    return pd.Series(d)

selected_lines = test_data.query('role =="manager"').groupby('dlg_id').apply(agg_func)
selected_lines = selected_lines.reset_index()
selected_lines = pd.melt(selected_lines, id_vars='dlg_id', value_name='line_n')

Оставим в test_data выбранные строки.

In [5]:
test_data = pd.merge(test_data, selected_lines, on=['dlg_id', 'line_n'])
test_data.drop(columns='variable', inplace=True)
test_data = test_data.sort_values(['dlg_id', 'line_n'])

with pd.option_context('display.max_colwidth', 100):
    display(test_data)

,dlg_id,line_n,role,text
0,0,1,manager,алло здравствуйте
1,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...
2,0,106,manager,и вам спасибо большое за обратную связь
3,0,108,manager,всего хорошего до свидания
4,1,1,manager,алло здравствуйте
5,1,2,manager,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила вн...
6,1,53,manager,угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего
7,1,54,manager,до свидания
8,2,2,manager,алло здравствуйте
9,2,3,manager,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы...


Чтобы извлечь имена сотрудников и названия компаний и лучше иметь базу с ФИО и полным/кратким наименованием компаний.

В задаче такая информация отсутствует, поэтому словарь имен получим с data.mos, а список компаний создадим.

In [6]:
# Источник - https://data.mos.ru/opendata/7704111479-svedeniya-o-naibolee-populyarnyh-mujskih-imenah-sredi-novorojdennyh
man_mames = pd.read_csv('data-6271-2022-08-26.zip', usecols=['Name'],
                        encoding='Windows-1251', sep=';',  skiprows=[1])
# Источник - https://data.mos.ru/opendata/7704111479-svedeniya-o-naibolee-populyarnyh-jenskih-imenah-sredi-novorojdennyh
woman_mames = pd.read_csv('data-6269-2022-08-26.zip', usecols=['Name'],
                        encoding='Windows-1251', sep=';',  skiprows=[1])

names = pd.concat([man_mames, woman_mames],
                  ignore_index=True).Name \
                                    .str.lower() \
                                    .unique().tolist()
print('Уникальных имен: ', len(names))

Уникальных имен:  406


Поиск проще проводить по одному отличительному слову и в последствии выдавать полное наименование (например, с ООО или ФГУП).

In [7]:
companies_shortname = ['китобизнес', 'диджитал']
companies_fullname = ['китобизнес', 'диджитал бизнес']
companies_dict  = {companies_shortname[i]: companies_fullname[i] for i in range(len(companies_shortname))}

# Извлечение имени менеджера и названия компании

Функция извлечения имени.

* Предполагаем что в строке обязательно должны присутствовать ключевые слова 'меня' и 'зовут' (MVP вариант), после чего извлекается имя менеджера.
* Из пятого диалога не парсим имя 'Анастасия', это сделано намеренно, менеджер не должен так представляться.

In [8]:
def find_name(text):
    try:
        name = ''
        if text.find('меня') != -1 and text.find('зовут') != -1:
            name = list(set(text.split()).intersection(names))[-1]
    except:
        name = ''
    return name

Функция для извлечения названия компании.

In [9]:
def find_company(text):
    try:
        name = list(
            set(text.split()).intersection(companies_dict.keys())
        )[0]
        return companies_dict[name]
    except:
        return ''

Результат извлечения данных.

In [10]:
test_data['manager'] = test_data.text.apply(find_name)
test_data['company'] = test_data.text.apply(find_company)
test_data

,dlg_id,line_n,role,text,manager,company
0,0,1,manager,алло здравствуйте,,
1,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес
2,0,106,manager,и вам спасибо большое за обратную связь,,
3,0,108,manager,всего хорошего до свидания,,
4,1,1,manager,алло здравствуйте,,
5,1,2,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес
6,1,53,manager,угу да вижу я эту почту хорошо тогда исправлю ...,,
7,1,54,manager,до свидания,,
8,2,2,manager,алло здравствуйте,,
9,2,3,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес


# Извлечение реплик

Зададим корпус для обучения.

In [11]:
hello = ['здравствуйте', 'приветствую', 'алло здравствуйте', 'здравствуйте приветствую вас'
         'добрый день', 'добрый вечер', 'доброе утро', 'привет', 'добрый']

bye = ['понедельника', 'вторника', 'среды', 'четверга', 'пятницы',
       'до понедельника', 'до вторника', 'до среды', 'до четверга', 'до пятницы',
       'до свидания', 'всего хорошего', 'до встречи', 'до завтра',
       'хорошего дня', 'хорошего вечера', 'всего доброго',
       'хороших выходных', 'все хорошо']

filler = ['вам высылаю', 'давайте диктуйте', 'вас понимаю', 'смотрите там',
          'да все хорошо', 'все будет хорошо', 'алло', 'поддержку чат', 'до этого',
          'до конца', 'до окончания', 'хорошо', 'добро', 'добрый человек',
          'много всего', 'хорошего настроения',
          'в понедельник', 'во вторник', 'в среду', 'в четверг', 'в пятницу']

train = pd.DataFrame(
    {
        'text': hello + bye + filler,
        'insight': ['h' for x in range(len(hello))] \
        + ['b' for x in range(len(bye))] \
        + ['f' for x in range(len(filler))]
    }
)

Векторизуем train, при этом сохраним только определённые ngram'ы.

In [12]:
tf = TfidfVectorizer(ngram_range=(1,2), use_idf=False)
tf.fit(train.text)

feature_names = tf.get_feature_names_out()

# Отельные слова такие как 'до', 'добрый' отсеим в remove_list.
remove_list = list(
    set(train.text.values.tolist()).symmetric_difference(feature_names))

# Зададим маску чтобы после преобразования убрать лишние ngram'ы.
mask = ~np.in1d(feature_names, remove_list)

# Получим признаки.
X = tf.transform(train.text)
X = X[:, mask]
X.shape

(48, 41)

Создадим train_set для CatBoost и обучим модель.

In [13]:
tf = TfidfVectorizer(ngram_range=(1,2))
train_set = Pool(tf.fit_transform(train.text),
                 label=train.insight)

params = {
    'loss_function': 'MultiClass',
    'depth': 2,
}

model = CatBoostClassifier(random_state=38, **params)
model.fit(train_set, verbose=0)

Теперь можно извлечь необходимые фразы.

In [14]:
test_data['insight'] = model.predict(tf.transform(test_data.text.values))
test_data

,dlg_id,line_n,role,text,manager,company,insight
0,0,1,manager,алло здравствуйте,,,h
1,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес,f
2,0,106,manager,и вам спасибо большое за обратную связь,,,f
3,0,108,manager,всего хорошего до свидания,,,b
4,1,1,manager,алло здравствуйте,,,h
5,1,2,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес,f
6,1,53,manager,угу да вижу я эту почту хорошо тогда исправлю ...,,,b
7,1,54,manager,до свидания,,,b
8,2,2,manager,алло здравствуйте,,,h
9,2,3,manager,меня зовут ангелина компания диджитал бизнес з...,ангелина,диджитал бизнес,f


Выполним проверку требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом».

In [15]:
def agg_polite(group):
    return pd.Series({'was_polite': set(group.insight.unique()).issuperset(['h','b'])})

test_data = pd.merge(test_data, test_data.groupby('dlg_id').apply(agg_polite), on='dlg_id')

Результат извлечения данных.

In [16]:
with pd.option_context('display.max_colwidth', 100):
    display(test_data.query('insight == "h" or insight == "b" or manager != ""'))

,dlg_id,line_n,role,text,manager,company,insight,was_polite
0,0,1,manager,алло здравствуйте,,,h,True
1,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серы...,ангелина,диджитал бизнес,f,True
3,0,108,manager,всего хорошего до свидания,,,b,True
4,1,1,manager,алло здравствуйте,,,h,True
5,1,2,manager,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила вн...,ангелина,диджитал бизнес,f,True
6,1,53,manager,угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего,,,b,True
7,1,54,manager,до свидания,,,b,True
8,2,2,manager,алло здравствуйте,,,h,True
9,2,3,manager,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы...,ангелина,диджитал бизнес,f,True
11,2,84,manager,все хорошо,,,b,True


# Вывод

* Создан скрипт для парсинга диалогов удовлетворяющий всем требованиям задания.
* Спорные моменты. Скрипт не определил 'да это анастасия' как приветствие.